In [1]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

/Users/kuoteng/.pyenv/versions/3.6.4/envs/NCKU-DSAI/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Parameters Config

In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [3]:
DATA_SIZE = 40000
TRAIN_SIZE = 30000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789+- '
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
EPOCH_SIZE = 3
LAYERS = 1

In [4]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [5]:
ctable = CharacterTable(chars)

In [6]:
ctable.indices_char

{0: ' ',
 1: '+',
 2: '-',
 3: '0',
 4: '1',
 5: '2',
 6: '3',
 7: '4',
 8: '5',
 9: '6',
 10: '7',
 11: '8',
 12: '9'}

# Data Generation

In [7]:
%time
questions = []
expected = []
seen = set()
print('Generating data...')

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.87 µs
Generating data...


In [8]:
%time
while len(questions) < DATA_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    a, b = (a, b) if a > b else (b, a)
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    q = '{}-{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a - b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.82 µs
Total addition questions: 40000


In [9]:
print(questions[:5], expected[:5])

['963-3  ', '911-20 ', '78-6   ', '873-31 ', '991-4  '] ['960 ', '891 ', '72  ', '842 ', '987 ']


# Processing

In [10]:
print('Vectorization... (to the one-hot encoding)')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization... (to the one-hot encoding)


In [11]:
indices = np.arange(len(y))
np.random.shuffle(indices)
print(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:TRAIN_SIZE]
train_y = y[:TRAIN_SIZE]
test_x = x[TRAIN_SIZE:]
test_y = y[TRAIN_SIZE:]

print('Training Data:')
print(train_x.shape)
print(train_y.shape)

split_at = len(train_x) - len(train_x) // 10
print('split_at', split_at)
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)


[22381 10416  3763 ...  6310 16992  5297]
Training Data:
(30000, 7, 13)
(30000, 4, 13)
split_at 27000
Training Data:
(27000, 7, 13)
(27000, 4, 13)
Validation Data:
(3000, 7, 13)
(3000, 4, 13)
Testing Data:
(10000, 7, 13)
(10000, 4, 13)


In [12]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False  True False False False False False False
   False False]
  [False False False False False False False False False  True False
   False False]
  [False False False False False False False False  True False False
   False False]
  [False False  True False False False False False False False False
   False False]
  [False False False False False  True False False False False False
   False False]
  [False False False False False False False False False False  True
   False False]
  [ True False False False False False False False False False False
   False False]]

 [[False False False False False False False  True False False False
   False False]
  [False False False False False False False False  True False False
   False False]
  [False False False  True False False False False False False False
   False False]
  [False False  True False False False False False False False False
   False False]
  [False False False False False False False False Fals

# Build Model

In [13]:
print('Build model...')
model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))
for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72704     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 13)             1677      
_________________________________________________________________
activation_1 (Activation)    (None, 4, 13)             0         
Total params: 205,965
Trainable params: 205,965
Non-trainable params: 0
_________________________________________________________________


# Training

In [14]:
%time
for loop in range(100):
    print()
    print('-' * 50)
    print('Train Loop Num:', loop)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=EPOCH_SIZE,
              validation_data=(x_val, y_val),
              shuffle=True)
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.06 µs

--------------------------------------------------
Train Loop Num: 0
Train on 27000 samples, validate on 3000 samples
Epoch 1/3
27000/27000 [==============================] - 7s 271us/step - loss: 1.8399 - acc: 0.3434 - val_loss: 1.7183 - val_acc: 0.3618
Epoch 2/3
27000/27000 [==============================] - 6s 223us/step - loss: 1.6371 - acc: 0.3829 - val_loss: 1.5892 - val_acc: 0.3991
Epoch 3/3
27000/27000 [==============================] - 6s 234us/step - loss: 1.5462 - acc: 0.4279 - val_loss: 1.5141 - val_acc: 0.4404
dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])
[0.3433888889418708, 0.3828796296737812, 0.42787037043218257]
[0.36183333333333334, 0.3990833332538605, 0.44041666650772093]
Q 892-12  T 880  ☒ 777 
Q 575-0   T 575  ☒ 555 
Q 862-91  T 771  ☒ 773 
Q 924-704 T 220  ☒ 304 
Q 789-192 T 597  ☒ 770 
Q 49-5    T 44   ☑ 44  
Q 33-20   T 13   ☒ 33  
Q 409-69  T 340  ☒ 335 
Q 991-506 T 485  ☒ 703 
Q 40-20   T 20 

KeyboardInterrupt: 

# Testing

In [ ]:
print("MSG : Prediction")
print("-" * 50)
right = 0
preds = model.predict_classes(test_x, verbose=0)
for i in range(len(preds)):
    q = ctable.decode(test_x[i])
    correct = ctable.decode(test_y[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    print('Q', q[::-1] if REVERSE else q, end=' ')
    print('T', correct, end=' ')
    if correct == guess:
        print(colors.ok + '☑' + colors.close, end=' ')
        right += 1
    else:
        print(colors.fail + '☒' + colors.close, end=' ')
    print(guess)
print("MSG : Accuracy is {}".format(right / len(preds)))

In [ ]:
# print("MSG : Prediction")
# test_x = ["555+275", "860+7  ", "340+29 "]
# test_y = ["830 ", "867 ", "369 "]
# x = np.zeros((len(test_x), MAXLEN, len(chars)), dtype=np.bool)
# y = np.zeros((len(test_y), DIGITS + 1, len(chars)), dtype=np.bool)
# for j, (i, c) in enumerate(zip(test_x, test_y)):
#     x[j] = ctable.encode(i, MAXLEN)
#     y[j] = ctable.encode(c, DIGITS + 1)